In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import xml.etree.ElementTree as ET
import numpy as np
import time
import os
import random
import xlwt

from utils import elem2sent

## Load model and metadata from disk

In [2]:
from ast import keyword


model = SentenceTransformer('distilbert-base-nli-mean-tokens')
meta_data_df = pd.read_excel('data/category_configuration_09-08-2022_08-08-01.xlsx', sheet_name = 'article_names_matching')
title2category = dict(zip(meta_data_df["Article Title"], meta_data_df["Category 2"]))

keyword_df = pd.read_excel('data/category_configuration_09-08-2022_08-08-01.xlsx', sheet_name = 'keywords_category_2_mapping')
#keyword_df["Category"] = keyword_df[['Category 3', 'Category 2']].apply(
#    lambda x: [x[0],x[1]],
#    axis=1)
keyword2category2 = dict(zip(keyword_df["Keyword"], keyword_df["Category 2"]))
keyword2category3 = dict(zip(keyword_df["Keyword"], keyword_df["Category 3"]))

## Code for calculate similarity between two documents

In [3]:
def get_article_alignment(all_articles1, all_articles2, title2category, sanity_check = True):
    '''
    Get article alignment between two documents
    article_body: xml element
    title2category: dictionary of title to category
    sanity_check: use similarity score to check if the alignment is correct
    return a dictionary of article alignment
    '''
    alignment_match = {}

    #filter out the arrticles that don't have title
    all_articles1 = [article for article in all_articles1 if article.get("title") is not None]
    all_articles2 = [article for article in all_articles2 if article.get("title") is not None]

    #if both articles attribute includes title
    #if 'title' in all_articles1[0].attrib and 'title' in all_articles2[0].attrib:
    article_title1 = [(article.get('title').lower(), article.get('num')) for article in all_articles1 if article.get('title')]
    article_title2 = [(article.get('title').lower(), article.get('num')) for article in all_articles2 if article.get('title')]
    for index1, (title1, num1) in enumerate(article_title1):
        for index2, (title2, num2) in enumerate(article_title2):
            if title2category.get(title1) == title2category.get(title2):
                #to make sure that they have a high similarity
                if sanity_check and max(util.cos_sim(
                    model.encode(''.join(elem2sent(all_articles1[index1]))),
                    model.encode(''.join(elem2sent(all_articles2[index2])))
                )).item() < 0.7:
                    continue
                alignment_match[num1] = num2
                break
                
    #title not included in article attri, use sentence similarity instead
    # else:
    #     for article1 in all_articles1:
    #         text1_embd = model.encode(''.join(elem2sent(article1)))
    #         text2_list_embd = model.encode([''.join(elem2sent(article2)) for article2 in all_articles2])
    #         scores = util.cos_sim(text1_embd, text2_list_embd)
    #         if max(scores[0]).item() > 0.7:
    #             index = np.argmax(scores[0])
    #             alignment_match[article1.get('num')] = all_articles2[index].get('num')

    return alignment_match
    

def extract_similar_sentences_from_article(article1, article2):
    '''
    article1, article2: xml element
    return: list of similar sentences and their category: (sentence1, sentence2, similarity), category
    '''
    if article1.get('title') and article2.get('title'):
        assert title2category.get(article1.get('title').lower()) == title2category.get(article2.get('title').lower())

    article1_sents, article2_sents = elem2sent(article1, break_sentence = False), elem2sent(article2, break_sentence = False)
    #article1_sents, article2_sents = elem2sent(article1), elem2sent(article2)

    #Embed article1 and article2
    article1_embd, article2_embd = model.encode(article1_sents), model.encode(article2_sents)

    #Get similarity between article1 and article2
    scores = util.cos_sim(article1_embd, article2_embd)

    visited = set() #to make sure that we don't add the same sentence twice

    #filter out the sentence with similarity greater than 0.98, this means they are perfect match and no need to compare
    identical = (scores > 0.98).to(torch.int64)
    for i, j in identical.nonzero().tolist():
        visited.add('row' + str(i))
        visited.add('col' + str(j))

    #filter out the sentences with similarity between 0.5 and 0.98
    mask = (scores > 0.5) & (scores < 0.98)
    scores *= mask.to(torch.int64) 

    #get the index of the sentences with similarity between 0.5 and 0.98
    sim_pairs = [(scores[i][j], i, j) for i, j in mask.nonzero().tolist()]
    sim_pairs.sort(key = lambda x: x[0]) #sort by similarity score
    
    ret = []
    while sim_pairs:
        score, i, j = sim_pairs.pop()
        if 'row' + str(i) not in visited and 'col' + str(j) not in visited:
            ret.append((article1_sents[i], article2_sents[j], scores[i][j].item()))
            visited.add('row' + str(i))
            visited.add('col' + str(j))

    return ret, title2category.get(article1.get('title').lower())

def extract_similar_from_doc(doc1_path, doc2_path, title2category, target_category = ['Fair and Equitable Treatment'], min_length = 5):
    '''
    doc_path: path to first document
    title2category: dictionary of title to category
    '''

    try:   
        doc1, doc2 = ET.parse(doc1_path), ET.parse(doc2_path)
        doc_root1, doc_root2 = doc1.getroot(), doc2.getroot()
        doc_body1, doc_body2 = doc_root1[1][2], doc_root2[1][2]
        
    except Exception as e:
        print(e); return []
    
    #get article alignment between two documents
    all_articles1, all_articles2 = doc_body1.findall(".//div[@type='article']"), doc_body2.findall(".//div[@type='article']")
    #alignment_match = get_article_alignment(doc_body1, doc_body2, title2category)
    alignment_match = get_article_alignment(all_articles1, all_articles2, title2category)
    #print(alignment_match)
    keywords = [k for k, v in keyword2category2.items() if v in target_category]

    ret = []
    for page1, page2 in alignment_match.items():
        article1, article2 = doc_body1.find(".//div[@num='" + page1 + "']"), doc_body2.find(".//div[@num='" + page2 + "']") 
        similar_sents, category = extract_similar_sentences_from_article(article1, article2)

        if category in target_category:
            for (sentence1, sentence2, score) in similar_sents:

                s1_cate3 = [keyword2category3.get(key) for key in keywords if key in sentence1.lower()]
                s2_cate3 = [keyword2category3.get(key) for key in keywords if key in sentence2.lower()]

                if set(s1_cate3).intersection(s2_cate3) :

                    ret.append((sentence1, sentence2, score, list(set(s1_cate3).intersection(s2_cate3))[0]))

    #filter out the pairs in which both sentences are longer than min_length words and length difference is less than 4 * min_length
    #ret = [x for x in ret if len(x[0].split()) > min_length and len(x[1].split()) > min_length and abs(len(x[0].split()) - len(x[1].split())) < 5 * min_length]
    ret = [x for x in ret if len(x[0].split()) > min_length and len(x[1].split()) > min_length]
    #sort by similarity score
    ret.sort(key = lambda x: x[2], reverse = True)

    return ret

Sanaity Check

In [5]:
start = time.time()
diff = extract_similar_from_doc(
    'data/full data/t1989-9-canada-russian-federation-bit-1989.xml', 
    'data/full data/t1990-14-canada-czech-republic-bit-1990.xml',
    title2category
)
print(time.time() - start)

C:\Users\73183\AppData\Local\Temp\ipykernel_33728\1730355243.py:65: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  for i, j in identical.nonzero().tolist():


9.908003091812134


Filtering out the documents that contain the target category

In [6]:
docs = ['data/full data/' + _ for _ in os.listdir('data/full data') ]
print("total treaty is " + str(len(docs)))
target_category = ['Fair and Equitable Treatment']
target_treaty = []

for treaty in docs:
    try:   
        doc1 = ET.parse(treaty)
        doc_root1 = doc1.getroot()
        doc_body1 = doc_root1[1][2]
        
    except Exception as e:
        continue

    for article in doc_body1:
        try:
            if title2category.get(article.get("title").lower()) in target_category:
                target_treaty.append(treaty)
                break
        except:
            continue

print(len(target_treaty))
    

total treaty is 3309
61


In [7]:
canada_docs = target_treaty

#Write all sentences to a excel file
wb = xlwt.Workbook()
ws = wb.add_sheet('sheet1')

#add header
ws.write(0, 0, 'sentence1')
ws.write(0, 1, 'sentence2')
ws.write(0, 2, 'similarity')
ws.write(0, 3, 'doc1')
ws.write(0, 4, 'doc2')
ws.write(0, 5, 'Subcategory')

row = 1

visited = set()

for i in range(100):
    doc1 = random.choice(canada_docs)
    doc1name = doc1.replace('.','/').split('/')[2]

    doc2 = random.choice(canada_docs)
    doc2name = doc2.replace('.','/').split('/')[2]

    if (doc1, doc2) in visited or (doc2, doc1) in visited:

        continue

    try:
        _ = extract_similar_from_doc(doc1, doc2, title2category, target_category)
        for j in range(len(_)):
            ws.write(row, 0, _[j][0])
            ws.write(row, 1, _[j][1])
            ws.write(row, 2, _[j][2])
            ws.write(row, 3, doc1name)
            ws.write(row, 4, doc2name)
            ws.write(row, 5, _[j][3])

            row += 1
            visited.add((doc1, doc2))

    except Exception as e:
        print(e)
        print(doc1, doc2)
        continue

wb.save('generated_data/similar_sentences.xls')


In [8]:
a = set({"a", "b", "c"})

"a" in a

True

Problematic sentences

In [9]:
doc1 = 't1991-3-france-united-arab-emirates-bit-1991'
doc2 = 't2015-28-haiti-mexico-bit-2015'

extract_similar_from_doc('data/full data/' + doc1 + '.xml', 'data/full data/' + doc2 + '.xml', title2category)

[('Each Contracting Party undertakes to ensure, in its territory and in its maritime zone, fair and equitable treatment, in accordance with the principles of international law, to investments of investors of the other Party, excluding any unfair or discriminatory measure which might hinder in law or in fact the management, maintenance, enjoyment or liquidation of such investments.',
  '1. Each Contracting Party shall accord to investments of investors as of the other contracting party treatment pursuant to the customary International Law including fair and equitable treatment and full protection and security.',
  0.8035207986831665,
  'Fair and Equitable Treatment')]